# Tiling into stripes

### table of content
1) 

### Dependencies and general utils

In [2]:
import os
import numpy as np
import pdal
import json
import laspy
from tqdm import tqdm

### code

In [6]:
src_input_file = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\Barmasse_2024_AllScans_Subsample4cm_temp.laz"
src_input_file = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\test_stripes\Barmasse_2024_AllScans_Subsample4cm_shifted.laz"
src_output = r"D:\GitHubProjects\Terranum_repo\TreeSegmentation\data\Barmasse\test_stripes"

tile_size_x, tile_size_y = 100, 20

In [4]:
laz = laspy.read(src_input_file)
print(np.min(laz.x))
print(np.max(laz.x))
print(np.min(laz.y))
print(np.max(laz.y))
delta_x = np.max(laz.x) - np.min(laz.x)
delta_y = np.max(laz.y) - np.min(laz.y)
print("---")
print(delta_x)
print(delta_y)
# tile_size_x = min(tile_size_x, delta_x)
# tile_size_y = min(tile_size_y, delta_y)

2587114.301854636
2588120.836854636
1098210.7911605174
1099375.1941605173
---
1006.535000000149
1164.402999999933


In [7]:
output_folder = os.path.join(src_output, f"stripes_{tile_size_x}_{tile_size_y}")
os.makedirs(output_folder, exist_ok=True)

xmin, xmax, ymin, ymax = np.min(laz.x), np.max(laz.x), np.min(laz.y), np.max(laz.y)

x_edges = np.arange(xmin, xmax, tile_size_x)
y_edges = np.arange(ymin, ymax, tile_size_y)


for i, x0 in enumerate(x_edges):
    print(f"Column {i+1} / {len(x_edges)}:")
    for j, y0 in tqdm(enumerate(y_edges), total=len(y_edges)):
        x1 = x0 + tile_size_x
        y1 = y0 + tile_size_y
        bounds = f"([{x0},{x1}],[{y0},{y1}])"
        output = os.path.join(output_folder,f"tile_{i}_{j}.laz")
        pipeline_json = {
            "pipeline": [
                src_input_file,
                {"type": "filters.crop", "bounds": bounds},
                {"type": "writers.las", "filename": output}
            ]
        }

        pipeline = pdal.Pipeline(json.dumps(pipeline_json))
        pipeline.execute()

Column 1 / 11:


  0%|          | 0/59 [01:07<?, ?it/s]


RuntimeError: filters.crop: Unable to reproject bounds.